In [1]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD= os.getenv('POSTGRES_PASSWORD')

In [2]:
engine = create_engine('postgresql+psycopg2://{user}:{pw}@{service}:{port}/{db}'.format(
    user = 'postgres',
    pw= POSTGRES_PASSWORD,
    service = 'postgres',
    port = '5432',
    db = 'nba'
    
))

In [3]:
myquery = '''
SELECT game_date, season, ot
FROM games
WHERE season=2022 AND ot>=2
ORDER BY ot DESC
LIMIT 20
   
'''
pd.read_sql_query(myquery, con = engine)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "games" does not exist
LINE 3: FROM games
             ^

[SQL: 
SELECT game_date, season, ot
FROM games
WHERE season=2022 AND ot>=2
ORDER BY ot DESC
LIMIT 20
   
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [11]:
# myquery = '''
# SELECT 
#     game_date AS date, 
#     season AS nba_season, 
#     ot AS OT
#     2*OT as two_times_ot
# FROM games
# WHERE season=2022 AND ot>=2
# ORDER BY ot DESC
# LIMIT 20
   
# '''
# pd.read_sql_query(myquery, con = engine)

#ER DIAGRAM HELPS TI VIEW TABLES WHEN DOING SQL

In [ ]:
#from the player game table, make a table with all players who scored more than 30 points in the game. Keep only the columns wuth the game and player id, points and three point shots made and attempted. Sort by points in descending order and display just the top 15

In [13]:
query= '''
SELECT
player_id, pts, fg3, fg3a
FROM player_game
WHERE pts>30
ORDER BY pts DESC
LIMIT 15


'''

pd.read_sql_query(query, con = engine)

,player_id,pts,fg3,fg3a
0,curryst01,62,8,16
1,lillada01,61,11,20
2,lillada01,61,9,17
3,lillada01,61,11,20
4,lillada01,61,9,17
5,hardeja01,60,8,14
6,bealbr01,60,7,10
7,tatumja01,60,5,7
8,lillada01,60,7,16
9,lillada01,60,7,16


In [15]:
myquery = '''
SELECT g.game_id, p.player, pg.pts, pg.fg3, pg.fg3a
FROM player_game pg
INNER JOIN players p
     ON pg.player_id = p.player_id
INNER JOIN games g
     ON pg.game_id = g.game_id
WHERE pg.pts > 30
ORDER BY pg.pts DESC
LIMIT 15
'''
pd.read_sql_query(myquery, con = engine)

,game_id,player,pts,fg3,fg3a
0,202101030GSW,Stephen Curry,62,8,16
1,202001200POR,Damian Lillard,61,11,20
2,202008110DAL,Damian Lillard,61,9,17
3,202001200POR,Damian Lillard,61,11,20
4,202008110DAL,Damian Lillard,61,9,17
5,201911300HOU,James Harden,60,8,14
6,202101060PHI,Bradley Beal,60,7,10
7,202104300BOS,Jayson Tatum,60,5,7
8,201911080POR,Damian Lillard,60,7,16
9,201911080POR,Damian Lillard,60,7,16


In the team game table who won each game. 
That will require you to join the team game to itself
Keep the game id, the team and opponent abbreviation, the team score, and the other teams score renamed to "Opponent Score"

In [17]:
query2 = '''
SELECT 
    tg.game_id, 
    tg.team_abbrev, 
    tg.opponent_abbrev, 
    tg.team_score, 
    tg1.team_score AS opponent_score,
    (tg.team_score > tg1.team_score) AS win
FROM team_game tg
INNER JOIN team_game tg1
    ON tg.game_id = tg1.game_id AND tg.opponent_abbrev = tg1.team_abbrev
'''
pd.read_sql_query(query2, con = engine)

,game_id,team_abbrev,opponent_abbrev,team_score,opponent_score,win
0,202202170BRK,WAS,BRK,117,103,True
1,202202170BRK,BRK,WAS,103,117,False
2,202202170CHO,MIA,CHO,111,107,True
3,202202170CHO,CHO,MIA,107,111,False
4,202202170LAC,HOU,LAC,111,142,False
...,...,...,...,...,...,...
6389,202002270GSW,GSW,LAL,86,116,False
6390,202002290PHO,GSW,PHO,115,99,True
6391,202003010GSW,GSW,WAS,110,124,False
6392,202003030DEN,GSW,DEN,116,100,True
